# Fake News detection 

### data handle lib

In [67]:
import pandas as pd
from sklearn.model_selection import train_test_split


### loading text processing librarys


In [68]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer


### Load Classifier

In [69]:
from sklearn.ensemble import (RandomForestClassifier,ExtraTreesClassifier,AdaBoostClassifier)
from sklearn.tree import DecisionTreeClassifier

### load data 

In [70]:
train_data=pd.read_csv("./fake-news/train.csv")
test_data=pd.read_csv("./fake-news/test.csv")


### start coding

In [71]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [72]:
# add label field in test to balance the test and trian data becouse trian missing label field
test_data['label']='t' 

test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5200 non-null   int64 
 1   title   5078 non-null   object
 2   author  4697 non-null   object
 3   text    5193 non-null   object
 4   label   5200 non-null   object
dtypes: int64(1), object(4)
memory usage: 203.2+ KB


In [73]:
# replace the null with " " 
test_data=test_data.fillna(" ")
train_data=train_data.fillna(" ")
# conbine the title, auther and text to get single field  
test_data['total']=test_data["title"]+" "+test_data['author']+" "+test_data["text"]
train_data['total']=train_data['title']+" "+train_data["author"]+" "+train_data["text"]

test_data.head()

,id,title,author,text,label,total
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",t,"Specter of Trump Loosens Tongues, if Not Purse..."
1,20801,Russian warships ready to strike terrorists ne...,,Russian warships ready to strike terrorists ne...,t,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,t,#NoDAPL: Native American Leaders Vow to Stay A...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...",t,"Tim Tebow Will Attempt Another Comeback, This ..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,t,Keiser Report: Meme Wars (E995) Truth Broadcas...


### CountVectorizer

it create token of word and count it number of time it appeared in doc  

In [74]:
count_vec=CountVectorizer(ngram_range=(1,2))
counts=count_vec.fit_transform(train_data['total'])


In [21]:
print(counts)

  (0, 1463243)	7
  (0, 850625)	3
  (0, 142619)	6
  (0, 3344934)	8
  (0, 887272)	8
  (0, 1058561)	5
  (0, 2697571)	4
  (0, 676947)	11
  (0, 1766993)	8
  (0, 3243162)	4
  (0, 1646520)	6
  (0, 593927)	15
  (0, 3200367)	3
  (0, 1623050)	16
  (0, 811641)	6
  (0, 1824065)	3
  (0, 522664)	2
  (0, 2162427)	12
  (0, 2113860)	2
  (0, 45954)	2
  (0, 34534)	2
  (0, 2912418)	1
  (0, 3016116)	41
  (0, 2908187)	1
  (0, 1511561)	15
  :	:
  (20799, 1500024)	2
  (20799, 369488)	1
  (20799, 2071771)	1
  (20799, 2033884)	1
  (20799, 2190229)	1
  (20799, 2218186)	1
  (20799, 2193748)	1
  (20799, 1805444)	1
  (20799, 1249980)	1
  (20799, 851209)	1
  (20799, 49082)	1
  (20799, 1588213)	1
  (20799, 2973921)	1
  (20799, 2213893)	1
  (20799, 49111)	1
  (20799, 798382)	1
  (20799, 598741)	1
  (20799, 829970)	1
  (20799, 2508001)	1
  (20799, 1298540)	1
  (20799, 2017909)	1
  (20799, 2887486)	1
  (20799, 49145)	1
  (20799, 1291832)	1
  (20799, 1465868)	1


### TfidfTransform
This summarizes how often a given word appears within a document


In [75]:
trasformer=TfidfTransformer(smooth_idf=False) # allow 0 division

tfidf=trasformer.fit_transform(counts)

do the same thing with test

In [76]:
test_count=count_vec.transform(test_data['total'])
test_tfidf=trasformer.fit_transform(test_count)

/home/redscorpian/anaconda3/envs/NLP/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:1466: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


splite the test and train

In [77]:
targets=train_data['label']
X_train,X_test,y_train,y_test=train_test_split(tfidf,targets,random_state=0)

    Classifications Logistic regression Decisiontree KNeighbours Linear Discriminant

In [34]:
extra_tree=ExtraTreesClassifier(n_estimators=5,n_jobs=4)
extra_tree.fit(X_train,y_train)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=4,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [36]:
print('Accuracy of ExtrTrees classifier on training set: {:.2f}'
     .format(extra_tree.score(X_train, y_train)))
print('Accuracy of Extratrees classifier on test set: {:.2f}'
     .format(extra_tree.score(X_test, y_test)))

Accuracy of ExtrTrees classifier on training set: 1.00
Accuracy of Extratrees classifier on test set: 0.84


In [38]:
Abad=AdaBoostClassifier(DecisionTreeClassifier(max_depth=3),n_estimators=5)
Abad.fit(X_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                          

In [82]:
print('Accuracy of Adaboost classifier on training set: {:.2f}'
     .format(Abad.score(X_train, y_train)))
print('Accuracy of Adaboost classifier on test set: {:.2f}'
     .format(Abad.score(X_test, y_test))) 

Accuracy of Adaboost classifier on training set: 0.96
Accuracy of Adaboost classifier on test set: 0.96


In [83]:
Adab=AdaBoostClassifier(DecisionTreeClassifier(max_depth=3),n_estimators=5)
targets=train_data['label']
Abad.fit(counts,targets)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                          

In [84]:
example_counts = count_vec.transform(test_data['total'])
pre=Abad.predict(example_counts)
preddata=pd.DataFrame(pre,columns=['label'])


In [86]:
preddata['id']=test_data['id']
preddata.groupby('label').count()

,id
label,
0,2588
1,2612


In [87]:
pred.to_csv('countpre.csv', index=False)

In [41]:
Rando= RandomForestClassifier(n_estimators=5)

Rando.fit(X_train, y_train)
print('Accuracy of randomforest classifier on training set: {:.2f}'
     .format(Rando.score(X_train, y_train)))
print('Accuracy of randomforest classifier on test set: {:.2f}'
     .format(Rando.score(X_test, y_test)))


Accuracy of randomforest classifier on training set: 0.99
Accuracy of randomforest classifier on test set: 0.82


In [42]:
from sklearn.naive_bayes import MultinomialNB

NB = MultinomialNB()
NB.fit(X_train, y_train)
print('Accuracy of NB  classifier on training set: {:.2f}'
     .format(NB.score(X_train, y_train)))
print('Accuracy of NB classifier on test set: {:.2f}'
     .format(NB.score(X_test, y_test)))

Accuracy of NB  classifier on training set: 0.88
Accuracy of NB classifier on test set: 0.79


In [78]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e5)
logreg.fit(X_train, y_train)
print('Accuracy of Lasso classifier on training set: {:.2f}'
     .format(logreg.score(X_train, y_train)))
print('Accuracy of Lasso classifier on test set: {:.2f}'
     .format(logreg.score(X_test, y_test)))

/home/redscorpian/anaconda3/envs/NLP/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy of Lasso classifier on training set: 1.00
Accuracy of Lasso classifier on test set: 0.98


In [79]:
targets=train_data['label']
logrega=LogisticRegression(C=1e5,max_iter=1000)
logrega.fit(counts,targets)



LogisticRegression(C=100000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [80]:
example_counts = count_vec.transform(test_data['total'])
predictions = logreg.predict(example_counts)
pred=pd.DataFrame(predictions,columns=['label'])

In [81]:
pred['id']=test_data['id']
pred.groupby('label').count()

,id
label,
0,4504
1,696


In [53]:
pred.to_csv('countpre.csv', index=False)

In [54]:
data1=pd.read_csv('./fake-news/submit.csv')

In [57]:
data1.groupby("label").count()

,id
label,
0,2339
1,2861
